In [ ]:
!git clone https://github.com/leonardozilli/FrameTrigger.git
%cd FrameTrigger/

In [1]:
!pip install transformers[torch] datasets -q

In [1]:
from src.fn17 import load_dataset_hf, load_dataset_nltk
from src.train import train, test
from src.predict import predict_triggers, predict_frames
from src.process_data import prepare_data

2024-02-12 12:49:09.074289: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-12 12:49:09.122718: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-12 12:49:09.986854: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
CHECKPOINT = 'bert-base-cased'
TASK = 'frames'

In [3]:
dataset = load_dataset_nltk()
tokenized_dataset = prepare_data(dataset, CHECKPOINT, task=TASK)
tokenized_dataset

[nltk_data] Downloading package framenet_v17 to /home/leo/nltk_data...
[nltk_data]   Package framenet_v17 is already up-to-date!
100%|██████████| 107/107 [00:20<00:00,  5.14it/s]


Map:   0%|          | 0/3425 [00:00<?, ? examples/s]

Map:   0%|          | 0/328 [00:00<?, ? examples/s]

Map:   0%|          | 0/1354 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3425
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 328
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1354
    })
})

In [7]:
from nltk.corpus import framenet as fn

id2label = {i: frame['name'] for i, frame in enumerate(fn.frames(), start=1)}
id2label[0] = "None"
label2id = {v: k for k, v in id2label.items()}

In [26]:
len(id2label)

1222

In [12]:
FRAME2ID = {}

for i, frame in enumerate(fn.frames(), start=1):
    FRAME2ID[f'B-{frame["name"]}'] = i * 2 - 1
    FRAME2ID[f'I-{frame["name"]}'] = i * 2

len(FRAME2ID)

2442

In [17]:
id2label = {}

for i, frame in enumerate(fn.frames(), start=1):
    id2label[i*2-1] = f'B-{frame["name"]}'
    id2label[i*2] = f'I-{frame["name"]}'

id2label[0] = "None"

label2id = {v: k for k, v in id2label.items()}

0

In [4]:
for el in dataset['train'][30]:
    print(dataset['train'][0][el])
for el in tokenized_dataset['train'][30]:
    print(tokenized_dataset['train'][0][el])

Jerusalem 's recorded history begins with its mention in Egyptian court records 4,000 years ago , but there had been human settlements here for centuries , probably millennia , before that .
['Jerusalem', "'s", 'recorded', 'history', 'begins', 'with', 'its', 'mention', 'in', 'Egyptian', 'court', 'records', '4,000', 'years', 'ago', ',', 'but', 'there', 'had', 'been', 'human', 'settlements', 'here', 'for', 'centuries', ',', 'probably', 'millennia', ',', 'before', 'that', '.']
['O', 'O', 'O', 'O', 'B-Process_start', 'O', 'O', 'B-Statement', 'O', 'O', 'B-Judicial_body', 'O', 'O', 'B-Measure_duration', 'B-Time_vector', 'O', 'O', 'B-Existence', 'O', 'I-Existence', 'B-People', 'B-Locale_by_use', 'O', 'O', 'B-Measure_duration', 'O', 'B-Likelihood', 'B-Calendric_unit', 'O', 'B-Time_vector', 'O', 'O']
[0, 0, 0, 0, 1747, 0, 0, 2155, 0, 0, 1275, 0, 0, 1423, 2275, 0, 0, 913, 0, 914, 1603, 1347, 0, 0, 1423, 0, 1325, 355, 0, 2275, 0, 0]
[101, 6167, 112, 188, 1802, 1607, 3471, 1114, 1157, 4734, 1107, 

In [ ]:
label2id['Existence']

In [13]:
tokenized_dataset['train'][0]

{'input_ids': [101,
  6167,
  112,
  188,
  1802,
  1607,
  3471,
  1114,
  1157,
  4734,
  1107,
  6210,
  2175,
  3002,
  125,
  117,
  1288,
  1201,
  2403,
  117,
  1133,
  1175,
  1125,
  1151,
  1769,
  7536,
  1303,
  1111,
  3944,
  117,
  1930,
  6159,
  1424,
  5813,
  117,
  1196,
  1115,
  119,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100,
  0,
  0,
  0,
  0,
  0,
  874,
  0,
  0,
  1078,
  0,
  0,
  638,
  0,
  0,
  0,
  0,
  712,
  1138,
  0,
  0,
  457,
  0,
  457,
  802,
  674,
  0,
  0,
  712,
  0,
  663,
  178,
  178,
  178,
  0,
  1138,
  0,
  0,
  -100]}

In [18]:
import importlib
import src.process_data
importlib.reload(src.process_data)
from src.process_data import prepare_data
tokenized_dataset = prepare_data(dataset, CHECKPOINT, task=TASK)
tokenized_dataset

Map:   0%|          | 0/3425 [00:00<?, ? examples/s]

Map:   0%|          | 0/328 [00:00<?, ? examples/s]

Map:   0%|          | 0/1354 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3425
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 328
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1354
    })
})

In [4]:
for e in dataset['train'][0]:
    print(dataset['train'][0][e])

Jerusalem 's recorded history begins with its mention in Egyptian court records 4,000 years ago , but there had been human settlements here for centuries , probably millennia , before that .
['Jerusalem', "'s", 'recorded', 'history', 'begins', 'with', 'its', 'mention', 'in', 'Egyptian', 'court', 'records', '4,000', 'years', 'ago', ',', 'but', 'there', 'had', 'been', 'human', 'settlements', 'here', 'for', 'centuries', ',', 'probably', 'millennia', ',', 'before', 'that', '.']
['O', 'O', 'O', 'O', 'B-Process_start', 'O', 'O', 'B-Statement', 'O', 'O', 'B-Judicial_body', 'O', 'O', 'B-Measure_duration', 'B-Time_vector', 'O', 'O', 'B-Existence', 'O', 'I-Existence', 'B-People', 'B-Locale_by_use', 'O', 'O', 'B-Measure_duration', 'O', 'B-Likelihood', 'B-Calendric_unit', 'O', 'B-Time_vector', 'O', 'O']
[-1, -1, -1, -1, 874, -1, -1, 1078, -1, -1, 638, -1, -1, 712, 1138, -1, -1, 457, -1, 457, 802, 674, -1, -1, 712, -1, 663, 178, -1, 1138, -1, -1]


In [5]:

for el in tokenized_dataset['train'][0]:
    print(tokenized_dataset['train'][0][el])

[101, 6167, 112, 188, 1802, 1607, 3471, 1114, 1157, 4734, 1107, 6210, 2175, 3002, 125, 117, 1288, 1201, 2403, 117, 1133, 1175, 1125, 1151, 1769, 7536, 1303, 1111, 3944, 117, 1930, 6159, 1424, 5813, 117, 1196, 1115, 119, 102]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[-100, -1, -1, -1, -1, -1, 874, -1, -1, 1078, -1, -1, 638, -1, -1, -1, -1, 712, 1138, -1, -1, 457, -1, 457, 802, 674, -1, -1, 712, -1, 663, 178, 178, 178, -1, 1138, -1, -1, -100]


## Train

In [4]:
OUT_DIR = './models'
N_EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 2e-5

In [ ]:
train(pretrained_model=CHECKPOINT, task=TASK, dataset=tokenized_dataset,
    epochs=N_EPOCHS, batch_size=BATCH_SIZE, lr=LEARNING_RATE, model_output_path=OUT_DIR)

## Test

In [ ]:
test(OUT_DIR, tokenized_dataset['test'], task=TASK)

## Predict

In [74]:
sentence = "When the moon hits your eye, that's 'amore'."
sentence2 = "The hallway smelt of boiled cabbage and old rag mats."

predict_triggers(pretrained_model=OUT_DIR, sentence=sentence)
predict_frames(pretrained_model=OUT_DIR, sentence=sentence, visualize=True)

"When* the moon hits* your eye* , that ' s ' amore ' ."